# Unidad 2 - Práctica Integradora
## Procesamiento y Análisis de Imágenes con OpenCV

**Nombre del estudiante:** [Tu nombre aquí]

**Fecha:** [Fecha de entrega]

---

## Introducción

Este notebook integra todos los conceptos vistos en la Unidad 2:
- Transformaciones de intensidad (brillo, contraste, gamma)
- Histogramas y ecualización
- Ruido sintético y filtrado espacial
- Operaciones morfológicas (binarización, erosión, dilatación, apertura, cierre)
- Transformaciones geométricas (rotación, resize, crop) - ver Unidad 1

**Instrucciones:**
1. Selecciona una o más imágenes de tu interés
2. Completa cada sección con comentarios explicando los resultados
3. Usa celdas de Markdown para documentar tus observaciones
4. Experimenta con diferentes parámetros y compara resultados

In [ ]:
# Importar bibliotecas necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("Bibliotecas importadas correctamente")
print(f"Versión de OpenCV: {cv2.__version__}")
print(f"Versión de NumPy: {np.__version__}")

## 1. Cargar y Visualizar Imagen Original

**Instrucciones:** Carga tu imagen y muestra sus propiedades básicas.

In [ ]:
# Cargar imagen (modifica la ruta según tu imagen)
img = cv2.imread("imagenes/DPP0357.TIF", cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Error: No se pudo cargar la imagen")
    print("Verifica la ruta del archivo")
else:
    # Información de la imagen
    print(f"Dimensiones: {img.shape}")
    print(f"Tipo de datos: {img.dtype}")
    print(f"Rango de valores: [{img.min()}, {img.max()}]")
    print(f"Tamaño en memoria: {img.nbytes / 1024:.2f} KB")
    
    # Visualizar
    plt.figure(figsize=(10,8))
    plt.imshow(img, cmap='gray')
    plt.title('Imagen Original', fontsize=14)
    plt.colorbar(label='Intensidad', shrink=0.8)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

### Comentarios sobre la imagen original:

*[Describe brevemente tu imagen: ¿Qué representa? ¿Está bien expuesta o es muy oscura/clara? ¿Tiene buen contraste?]*

---

## 2. Transformaciones de Intensidad y Contraste

### 2.1 Ajuste Lineal (Brillo y Contraste)

**Fórmula:** $g(x,y) = \alpha \cdot f(x,y) + \beta$

Donde:
- $\alpha$ controla el contraste
- $\beta$ controla el brillo

In [ ]:
# Experimenta con diferentes valores de alpha y beta
alpha = 1.3  # Contraste (prueba valores entre 0.5 y 2.0)
beta = 30    # Brillo (prueba valores entre -50 y 50)

img_ajustada = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

# Visualización comparativa
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(img_ajustada, cmap='gray')
plt.title(f'Ajustada (α={alpha}, β={beta})')
plt.axis('off')

# Diferencia
plt.subplot(1, 3, 3)
diferencia = cv2.absdiff(img, img_ajustada)
plt.imshow(diferencia, cmap='hot')
plt.title('Diferencia (mapa de calor)')
plt.colorbar(shrink=0.8)
plt.axis('off')

plt.tight_layout()
plt.show()

**Observaciones:**

*[¿Qué cambios observas con estos valores de α y β? ¿Mejora el contraste? ¿Hay saturación de píxeles?]*

---

### 2.2 Corrección Gamma (Transformación No Lineal)

**Fórmula:** $g(x,y) = 255 \cdot \left(\frac{f(x,y)}{255}\right)^\gamma$

- $\gamma < 1$: Aclara la imagen
- $\gamma > 1$: Oscurece la imagen

In [ ]:
def ajustar_gamma(imagen, gamma=1.0):
    """Aplica corrección gamma a una imagen"""
    lut = np.array([((i / 255.0) ** gamma) * 255 
                    for i in range(256)]).astype("uint8")
    return cv2.LUT(imagen, lut)

# Probar diferentes valores de gamma
gamma_values = [0.5, 1.0, 2.0]

plt.figure(figsize=(15, 5))
for i, gamma in enumerate(gamma_values):
    img_gamma = ajustar_gamma(img, gamma)
    plt.subplot(1, 3, i+1)
    plt.imshow(img_gamma, cmap='gray')
    plt.title(f'Gamma = {gamma}')
    plt.axis('off')

plt.tight_layout()
plt.show()

**Observaciones:**

*[Compara los resultados con diferentes gammas. ¿Cuál funciona mejor para tu imagen? ¿Por qué?]*

---

## 3. Análisis de Histogramas

### 3.1 Histograma Original vs Ecualizado

In [ ]:
# Calcular histograma
hist_original = cv2.calcHist([img], [0], None, [256], [0,256])

# Aplicar ecualización
img_equalizada = cv2.equalizeHist(img)
hist_equalizada = cv2.calcHist([img_equalizada], [0], None, [256], [0,256])

# Visualización
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Fila 1: Imágenes
axes[0,0].imshow(img, cmap='gray')
axes[0,0].set_title('Original')
axes[0,0].axis('off')

axes[0,1].imshow(img_equalizada, cmap='gray')
axes[0,1].set_title('Ecualizada')
axes[0,1].axis('off')

axes[0,2].imshow(cv2.absdiff(img, img_equalizada), cmap='hot')
axes[0,2].set_title('Diferencia')
axes[0,2].axis('off')

# Fila 2: Histogramas
axes[1,0].bar(range(256), hist_original.ravel(), color='gray', alpha=0.7)
axes[1,0].set_title('Histograma Original')
axes[1,0].set_xlabel('Intensidad')
axes[1,0].set_ylabel('Frecuencia')
axes[1,0].set_xlim([0, 256])

axes[1,1].bar(range(256), hist_equalizada.ravel(), color='blue', alpha=0.7)
axes[1,1].set_title('Histograma Ecualizado')
axes[1,1].set_xlabel('Intensidad')
axes[1,1].set_ylabel('Frecuencia')
axes[1,1].set_xlim([0, 256])

# Comparación de histogramas
axes[1,2].plot(hist_original, color='gray', alpha=0.7, label='Original')
axes[1,2].plot(hist_equalizada, color='blue', alpha=0.7, label='Ecualizada')
axes[1,2].set_title('Comparación')
axes[1,2].set_xlabel('Intensidad')
axes[1,2].set_ylabel('Frecuencia')
axes[1,2].legend()

plt.tight_layout()
plt.show()

**Observaciones:**

*[¿Mejora el contraste con la ecualización? ¿En qué regiones de la imagen es más notorio? ¿El histograma ecualizado distribuye mejor las intensidades?]*

---

## 4. Filtrado Espacial y Eliminación de Ruido

### 4.1 Generar Ruido Sintético

In [ ]:
def agregar_ruido_gaussiano(imagen, sigma=25):
    """Agrega ruido gaussiano"""
    ruido = np.random.normal(0, sigma, imagen.shape)
    ruidosa = np.clip(imagen.astype(np.float32) + ruido, 0, 255)
    return ruidosa.astype(np.uint8)

def agregar_ruido_sp(imagen, prob=0.02):
    """Agrega ruido sal y pimienta"""
    ruidosa = imagen.copy()
    total = imagen.size
    
    # Sal
    num_sal = int(prob * total)
    coords = [np.random.randint(0, i, num_sal) for i in imagen.shape]
    ruidosa[coords[0], coords[1]] = 255
    
    # Pimienta
    num_pimienta = int(prob * total)
    coords = [np.random.randint(0, i, num_pimienta) for i in imagen.shape]
    ruidosa[coords[0], coords[1]] = 0
    
    return ruidosa

# Generar imágenes con ruido
np.random.seed(42)
img_gauss = agregar_ruido_gaussiano(img, sigma=25)
img_sp = agregar_ruido_sp(img, prob=0.02)

# Visualización
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(img, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(img_gauss, cmap='gray')
plt.title('Ruido Gaussiano (σ=25)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(img_sp, cmap='gray')
plt.title('Ruido Sal y Pimienta (2%)')
plt.axis('off')

plt.tight_layout()
plt.show()

### 4.2 Aplicar y Comparar Filtros

In [ ]:
# Tamaño de kernel
k = 5

# Filtros sobre ruido gaussiano
gauss_promedio = cv2.blur(img_gauss, (k, k))
gauss_gaussiano = cv2.GaussianBlur(img_gauss, (k, k), 0)
gauss_mediana = cv2.medianBlur(img_gauss, k)

# Filtros sobre ruido sal y pimienta
sp_promedio = cv2.blur(img_sp, (k, k))
sp_gaussiano = cv2.GaussianBlur(img_sp, (k, k), 0)
sp_mediana = cv2.medianBlur(img_sp, k)

# Visualización
fig, axes = plt.subplots(2, 5, figsize=(18, 8))

# Fila 1: Ruido Gaussiano
axes[0,0].imshow(img, cmap='gray'); axes[0,0].set_title('Original'); axes[0,0].axis('off')
axes[0,1].imshow(img_gauss, cmap='gray'); axes[0,1].set_title('Con Ruido Gaussiano'); axes[0,1].axis('off')
axes[0,2].imshow(gauss_promedio, cmap='gray'); axes[0,2].set_title('Filtro Promedio'); axes[0,2].axis('off')
axes[0,3].imshow(gauss_gaussiano, cmap='gray'); axes[0,3].set_title('Filtro Gaussiano'); axes[0,3].axis('off')
axes[0,4].imshow(gauss_mediana, cmap='gray'); axes[0,4].set_title('Filtro Mediana'); axes[0,4].axis('off')

# Fila 2: Ruido Sal y Pimienta
axes[1,0].imshow(img, cmap='gray'); axes[1,0].set_title('Original'); axes[1,0].axis('off')
axes[1,1].imshow(img_sp, cmap='gray'); axes[1,1].set_title('Con Ruido S&P'); axes[1,1].axis('off')
axes[1,2].imshow(sp_promedio, cmap='gray'); axes[1,2].set_title('Filtro Promedio'); axes[1,2].axis('off')
axes[1,3].imshow(sp_gaussiano, cmap='gray'); axes[1,3].set_title('Filtro Gaussiano'); axes[1,3].axis('off')
axes[1,4].imshow(sp_mediana, cmap='gray'); axes[1,4].set_title('Filtro Mediana'); axes[1,4].axis('off')

plt.suptitle(f'Comparación de Filtros (kernel {k}×{k})', fontsize=14)
plt.tight_layout()
plt.show()

**Observaciones:**

*[¿Qué filtro es más adecuado para cada tipo de ruido? ¿Por qué el filtro de mediana funciona mejor para sal y pimienta?]*

---

## 5. Operaciones Morfológicas

### 5.1 Binarización

In [ ]:
# Binarización con Otsu
umbral_otsu, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print(f"Umbral calculado por Otsu: {umbral_otsu:.2f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 3, 1)
plt.imshow(img, cmap='gray')
plt.title('Escala de Grises')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(binary, cmap='gray')
plt.title(f'Binarizada (Otsu, umbral={umbral_otsu:.0f})')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.hist(img.ravel(), bins=256, color='gray', alpha=0.7)
plt.axvline(x=umbral_otsu, color='red', linestyle='--', linewidth=2, label='Umbral Otsu')
plt.title('Histograma')
plt.xlabel('Intensidad')
plt.ylabel('Frecuencia')
plt.legend()

plt.tight_layout()
plt.show()

### 5.2 Erosión, Dilatación, Apertura y Cierre

In [ ]:
# Crear elemento estructurante
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

# Aplicar operaciones morfológicas
erosion = cv2.erode(binary, kernel, iterations=1)
dilatacion = cv2.dilate(binary, kernel, iterations=1)
apertura = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
cierre = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
gradiente = cv2.morphologyEx(binary, cv2.MORPH_GRADIENT, kernel)

# Visualización
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

operaciones = [
    (binary, 'Original Binaria'),
    (erosion, 'Erosión'),
    (dilatacion, 'Dilatación'),
    (apertura, 'Apertura\n(Erosión + Dilatación)'),
    (cierre, 'Cierre\n(Dilatación + Erosión)'),
    (gradiente, 'Gradiente Morfológico')
]

for i, (imagen, titulo) in enumerate(operaciones):
    axes[i].imshow(imagen, cmap='gray')
    axes[i].set_title(titulo, fontsize=12)
    axes[i].axis('off')

plt.suptitle('Operaciones Morfológicas (kernel 5×5)', fontsize=14)
plt.tight_layout()
plt.show()

**Observaciones:**

*[Describe el efecto de cada operación morfológica. ¿Cómo afectan a los objetos y al ruido? ¿Para qué casos usarías cada una?]*

---

### 5.3 Ejemplo Práctico: Limpieza de Imagen Binaria con Ruido

In [ ]:
# Agregar ruido sal y pimienta a la imagen binaria
binary_ruidosa = agregar_ruido_sp(binary, prob=0.02)

# Limpiar: primero apertura (elimina sal), luego cierre (elimina pimienta)
limpia_paso1 = cv2.morphologyEx(binary_ruidosa, cv2.MORPH_OPEN, kernel)
limpia_final = cv2.morphologyEx(limpia_paso1, cv2.MORPH_CLOSE, kernel)

# Visualización del proceso
plt.figure(figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.imshow(binary, cmap='gray')
plt.title('Original Limpia')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(binary_ruidosa, cmap='gray')
plt.title('Con Ruido S&P (2%)')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(limpia_paso1, cmap='gray')
plt.title('Después de Apertura')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(limpia_final, cmap='gray')
plt.title('Después de Cierre\n(limpia)')
plt.axis('off')

plt.tight_layout()
plt.show()

# Métricas
diferencia = cv2.absdiff(binary, limpia_final)
print(f"Píxeles diferentes después de limpieza: {np.sum(diferencia > 0)}")
print(f"Porcentaje de recuperación: {100 * (1 - np.sum(diferencia > 0) / binary.size):.2f}%")

**Observaciones:**

*[¿La combinación de apertura y cierre logra limpiar el ruido? ¿Qué porcentaje de la imagen se recuperó correctamente?]*

---

## 6. Transformaciones Geométricas (Opcional - Ver Unidad 1)

Si quieres incluir transformaciones geométricas, puedes agregar rotación, resize y crop aquí.
Consulta el material de la Unidad 1 para ejemplos completos.

In [ ]:
# Rotación
h, w = img.shape
centro = (w // 2, h // 2)
M_rot = cv2.getRotationMatrix2D(centro, 15, 1.0)  # 15 grados
img_rotada = cv2.warpAffine(img, M_rot, (w, h))

# Cambio de tamaño
img_pequena = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
img_grande = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)

# Recorte (crop)
recorte = img[100:400, 100:400]

# Visualización
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.imshow(img, cmap='gray')
plt.title(f'Original ({img.shape})')
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(img_rotada, cmap='gray')
plt.title('Rotada 15°')
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(recorte, cmap='gray')
plt.title(f'Recortada ({recorte.shape})')
plt.axis('off')

plt.subplot(2, 3, 5)
plt.imshow(img_pequena, cmap='gray')
plt.title(f'Reducida 50% ({img_pequena.shape})')
plt.axis('off')

plt.subplot(2, 3, 6)
plt.imshow(img_grande, cmap='gray')
plt.title(f'Ampliada 150% ({img_grande.shape})')
plt.axis('off')

plt.tight_layout()
plt.show()

**Observaciones:**

*[¿En qué problemas sería importante alinear la imagen o normalizar su tamaño antes de extraer características?]*

---

## 7. Resumen y Conclusiones

### Resumen de operaciones realizadas:

1. **Transformaciones de intensidad:**
   - Ajuste lineal (brillo/contraste)
   - Corrección gamma

2. **Histogramas:**
   - Análisis de distribución de intensidades
   - Ecualización de histograma

3. **Filtrado espacial:**
   - Generación de ruido gaussiano y sal y pimienta
   - Filtros: promedio, gaussiano, mediana

4. **Operaciones morfológicas:**
   - Binarización con Otsu
   - Erosión, dilatación, apertura, cierre
   - Limpieza de ruido en imágenes binarias

5. **Transformaciones geométricas (opcional):**
   - Rotación, resize, crop

### Conclusiones principales:

*[Escribe tus conclusiones principales aquí:]*

1. *[¿Qué transformación de intensidad fue más útil para tu imagen?]*
2. *[¿Qué filtro funcionó mejor para cada tipo de ruido?]*
3. *[¿Cómo ayudaron las operaciones morfológicas a limpiar la imagen?]*
4. *[¿Qué aprendiste sobre el preprocesamiento de imágenes?]*

---

## Referencias

- Gonzalez, R. C., & Woods, R. E. (2018). *Digital Image Processing* (4th ed.). Pearson.
- OpenCV Documentation: https://docs.opencv.org/4.x/
- Material del curso de Tratamiento Digital de Imágenes - CIMAT/INFOTEC